In [6]:
# For adding  flights information in flights table in sql database
import functions_framework
import pandas as pd
import requests
from datetime import datetime, timedelta
from pytz import timezone
from keys import RAPIAPI_KEY, sql_password

@functions_framework.http
def retrive_and_save_flights_data(request):
    connection_string = create_connection_string()
    airports_data = read_airports_data_sql(connection_string)
    df_flights_info = fetch_flights_data(airports_data)
    df_flights_info.to_sql('flights',
                           if_exists='append',
                           con=connection_string,
                           index=False)
    return "Flight information retrived and saved in database"
    
#function to create connection string to make connection with Database
#function to create connection string to make connection with Database
def create_connection_string():
  schema = "gans"
  host = "35.241.135.102"
  user = "root"
  password = sql_password
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

#to read airports information from sql
def read_airports_data_sql(connection_string):
  return pd.read_sql("airports", con=connection_string)

# to fetch flight information for individual airport with icao = icao_arrival
def get_airport_flights_data(icao_arrival, timezone_str):
    # your code here
    
    berlin_timezone = timezone('Europe/Berlin')
    
    airport_timezone = timezone(timezone_str)
    today = datetime.now(airport_timezone).date()
    tomorrow = (today + timedelta(days=1))
    time = ["08:00","20:00"]
 
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao_arrival}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

    querystring = {"withLeg":"true",
                   "direction":"Arrival",
                   "withCancelled":"false",
                   "withCodeshared":"true",
                   "withCargo":"false",
                   "withPrivate":"false"}
    headers = {
    	"x-rapidapi-key": RAPIAPI_KEY,
    	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code ==200:
        response_json = response.json()
        arrival_items = response_json['arrivals']
        scheduled_arrival_time =[]
        flight_status = [] 
        terminal = []
        flight_number = []
        arrival_airport_icao = []

        retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

        for item in arrival_items:
            scheduled_arrival_time.append(item['arrival']['scheduledTime']['local'])
            flight_status.append(item['status'])
            terminal.append(item['arrival'].get('terminal'))
            flight_number.append(item['number'])
            arrival_airport_icao.append(icao_arrival)

        flights_df = pd.DataFrame({"arrival_airport_icao":arrival_airport_icao,
                                   "flight_number":flight_number,
                                   "scheduled_arrival_time":scheduled_arrival_time,
                                   "flight_status":flight_status,
                                   'terminal':terminal,
                                   "data_retrieved_at": retrieval_time})
        flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
        flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
        flights_df["data_retrieved_at"] = pd.to_datetime(flights_df["data_retrieved_at"])
        return flights_df
    else:
        return 

# to fetch flight data for all airports in the airport table
def fetch_flights_data(airports_data):
    flights_info = []
    for index, row in airports_data.iterrows():
        iceo_arrival = row['icao']
        timezone_str = row['timeZone']
        flights_data = get_airport_flights_data(iceo_arrival,timezone_str)
        flights_info.append(flights_data)

    df_flights_info = pd.concat(flights_info, ignore_index=True)

    return df_flights_info






In [10]:
from flask import Request
import json

request_data = {}
request = Request.from_values(data=json.dumps(request_data))

# Call the function
response = retrive_and_save_flights_data(request)
print(response)

Flight information retrived and saved in database
